In [3]:
from datasets import load_dataset

dataset = load_dataset("sms_spam", split="train")
dataset = dataset.train_test_split(test_size=0.2, seed=42)

print(dataset)
print(dataset["train"][0])



/opt/miniconda3/envs/hf-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})
{'sms': 'K..then come wenever u lik to come and also tel vikky to come by getting free time..:-)\n', 'label': 0}


In [4]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    return tokenizer(batch["sms"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(preprocess, batched=True)


/opt/miniconda3/envs/hf-env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 1115/1115 [00:00<00:00, 13068.92 examples/s]


In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
)



/opt/miniconda3/envs/hf-env/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

                                                 
 50%|█████     | 279/558 [02:32<02:28,  1.88it/s]

{'eval_loss': 0.04152942821383476, 'eval_accuracy': 0.989237668161435, 'eval_precision': 0.9790209790209791, 'eval_recall': 0.9395973154362416, 'eval_f1': 0.958904109589041, 'eval_runtime': 11.1065, 'eval_samples_per_second': 100.392, 'eval_steps_per_second': 6.303, 'epoch': 1.0}


 90%|████████▉ | 500/558 [04:22<00:29,  2.00it/s]

{'loss': 0.0461, 'grad_norm': 0.027527669444680214, 'learning_rate': 5.197132616487455e-06, 'epoch': 1.79}


                                                 
100%|██████████| 558/558 [05:05<00:00,  1.83it/s]

{'eval_loss': 0.055490873754024506, 'eval_accuracy': 0.9901345291479821, 'eval_precision': 0.9928571428571429, 'eval_recall': 0.9328859060402684, 'eval_f1': 0.9619377162629758, 'eval_runtime': 9.7603, 'eval_samples_per_second': 114.238, 'eval_steps_per_second': 7.172, 'epoch': 2.0}
{'train_runtime': 305.7168, 'train_samples_per_second': 29.171, 'train_steps_per_second': 1.825, 'train_loss': 0.04191495451448639, 'epoch': 2.0}


TrainOutput(global_step=558, training_loss=0.04191495451448639, metrics={'train_runtime': 305.7168, 'train_samples_per_second': 29.171, 'train_steps_per_second': 1.825, 'total_flos': 295336065303552.0, 'train_loss': 0.04191495451448639, 'epoch': 2.0})

In [ ]:
results = trainer.evaluate()
print(results)


100%|██████████| 70/70 [00:09<00:00,  7.31it/s]

{'eval_loss': 0.055490873754024506, 'eval_accuracy': 0.9901345291479821, 'eval_precision': 0.9928571428571429, 'eval_recall': 0.9328859060402684, 'eval_f1': 0.9619377162629758, 'eval_runtime': 10.7937, 'eval_samples_per_second': 103.301, 'eval_steps_per_second': 6.485, 'epoch': 2.0}


In [ ]:
import os
import torch

# optional: enable MPS fallback
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "1")

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Device:", device)

model.to(device)
model.eval()

text = "Congratulations! You have won a free iPhone."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)

pred = outputs.logits.argmax(dim=1).item()
print("Prediction:", "spam" if pred == 1 else "ham")


Device: mps
Prediction: ham
